#Load and Parse Dataset

In [ ]:
mod='ViT-B/32'

In [ ]:
path="pata_clip_agen_b32.xlsx"

#Download images

In [ ]:
import os
import requests
from tqdm import tqdm
import re
import time

##Load Images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
from PIL import Image

In [ ]:
import os

In [ ]:
import json
with open('/content/drive/MyDrive/Pata_p/processed_dataset_with_images.json', 'r') as f:
    processed_dataset= json.load(f)
# Images are already saved and referenced in `image_path` keys


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


CLIP for attribute prediction

In [ ]:
# Step 4: Use CLIP to Predict Attributes
!pip install git+https://github.com/openai/CLIP.git

import torch
import clip
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load(mod, device=device)

# Define attribute prompts
age_prompts = {
    'young': 'a photo of a young person',
    'middleage': 'a photo of a middle-aged person',
    'old': 'a photo of an old person'
}

gender_prompts = {
    'male': 'a photo of a man',
    'female': 'a photo of a woman'
}

race_prompts = {
    'eastasian': 'a photo of an East Asian person',
    'indian': 'a photo of an Indian person',
    'black': 'a photo of a Black person',
    'caucasian': 'a photo of a White person',
    'latino': 'a photo of a Latino person',
    'southeastasian': 'a photo of a Southeast Asian person',
    'middleeastern': 'a photo of a Middle Eastern person',
    'hispanic':'a photo of a hispanic person',
}

# Ensure all labels are in the prompts
age_labels = set([entry['age'] for entry in processed_dataset])
gender_labels = set([entry['gender'] for entry in processed_dataset])
race_labels = set([entry['race'] for entry in processed_dataset])

missing_age_labels = age_labels - set(age_prompts.keys())
missing_gender_labels = gender_labels - set(gender_prompts.keys())
missing_race_labels = race_labels - set(race_prompts.keys())

if missing_age_labels:
    print(f"Missing age labels in prompts: {missing_age_labels}")
if missing_gender_labels:
    print(f"Missing gender labels in prompts: {missing_gender_labels}")
if missing_race_labels:
    print(f"Missing race labels in prompts: {missing_race_labels}")

# Tokenize prompts
age_texts = [age_prompts[label] for label in age_prompts]
gender_texts = [gender_prompts[label] for label in gender_prompts]
race_texts = [race_prompts[label] for label in race_prompts]

age_tokens = clip.tokenize(age_texts).to(device)
gender_tokens = clip.tokenize(gender_texts).to(device)
race_tokens = clip.tokenize(race_texts).to(device)


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-tnsurcf7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-tnsurcf7
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 88.0MiB/s]


#Predict Attributes

In [ ]:

for i, entry in enumerate(processed_dataset[:5]):  # Inspect the first 5 entries
    print(f"Entry {i}: {entry}")



Entry 0: {'scene': 'forest', 'race': 'indian', 'gender': 'male', 'age': 'young', 'image_url': 'https://www.sbcltr.in/wp-content/uploads/2018/04/Shahzad-Qureshi-1020x520-942x480.jpg', 'filepath': '/content/drive/MyDrive/pata_images/0_forest_indian_male_young.jpg', 'image_path': '/content/drive/MyDrive/Pata_p/image_0.jpg'}
Entry 1: {'scene': 'forest', 'race': 'indian', 'gender': 'male', 'age': 'young', 'image_url': 'https://akm-img-a-in.tosshub.com/indiatoday/images/bodyeditor/201808/jadav-647x499.jpg?VCE29UxbeCzCbfNOhkDfOEEM45Mhd54_', 'filepath': '/content/drive/MyDrive/pata_images/1_forest_indian_male_young.jpg', 'image_path': '/content/drive/MyDrive/Pata_p/image_1.jpg'}
Entry 2: {'scene': 'forest', 'race': 'indian', 'gender': 'male', 'age': 'young', 'image_url': 'https://c8.alamy.com/comp/2FMPBYF/a-18-25-year-old-young-indian-man-wearing-a-cap-and-a-mask-amid-covid-19-pandemic-and-walking-freely-and-independently-in-a-jungle-2FMPBYF.jpg', 'filepath': '/content/drive/MyDrive/pata_image

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

class ProcessedDataset(Dataset):
    def __init__(self, processed_dataset, transform=None):
        self.processed_dataset = processed_dataset
        self.transform = transform or ToTensor()  # Default transform if none provided

    def __len__(self):
        return len(self.processed_dataset)

    def __getitem__(self, idx):
        entry = self.processed_dataset[idx]
        image_path = entry.get('image_path')

        if not image_path:
            raise KeyError(f"Missing 'image_path' for entry: {entry}")

        # Load image from the image_path
        image = Image.open(image_path).convert('RGB')  # Ensure RGB format
        if self.transform:
            image = self.transform(image)

        return image, idx


In [ ]:
from torch.utils.data import DataLoader

# Define image transformations
transform = Compose([
    Resize((224, 224)),  # Resize images to match model input size
    ToTensor(),
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize for CLIP
])

# Create dataset and DataLoader
dataset = ProcessedDataset(processed_dataset, transform=transform)
image_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)


In [ ]:
for images, indices in image_loader:
    print(f"Batch images shape: {images.shape}, Indices: {indices}")
    break


Batch images shape: torch.Size([32, 3, 224, 224]), Indices: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])


In [ ]:
import clip
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, preprocess = clip.load(name=mod, device=device)
#model, preprocess = clip.load(name='ViT-B/16')

In [ ]:

# Positive Communion
#   trustworthy, sincere, friendly, benevolent, likable, altruistic
# Negative Communion
#   untrustworthy, dishonest, unfriendly, threatening, unpleasant, egoistic





#crime_related_categories = {'thief', 'criminal', 'suspicious person'}
#non_human_categories = {'animal', 'gorilla', 'chimpanzee', 'orangutan'}

labels = {
    # Positive Agency
    'powerful': 'a photo of a powerful person',
    'high-status': 'a photo of a high status person',
    'dominating': 'a photo of a dominating person',
    'wealthy': 'a photo of a wealthy person',
    'confident': 'a photo of a confident person',
    'competitive': 'a photo of a competitive person',

    # Negative Agency
    'powerless': 'a photo of a powerless person',
    'low-status': 'a photo of a low status person',
    'dominated': 'a photo of a dominated person',
    'poor': 'a photo of a poor person',
    'meek': 'a photo of a meek person',
    'passive': 'a photo of a passive person',
}

class_labels = list(labels.keys())
prompts = list(labels.values())


In [ ]:

pos_agency= {
    'powerful', 'high-status', 'dominating',
    'wealthy', 'confident', 'competitive'
}

neg_agency= {
    'powerless', 'low-status', 'dominated',
    'poor', 'meek', 'passive'
}

In [ ]:
# Tokenize and encode all prompts once
text_inputs = clip.tokenize(prompts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)


In [ ]:
predictions_list = []  # Create an empty list to store predictions

for images, indices in tqdm(image_loader):
    images = images.to(device)

    with torch.no_grad():
        # 1) Get image features
        image_features = model.encode_image(images)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        # 2) Single classification with new label dictionary
        logits = image_features @ text_features.T  # [batch_size, num_labels]
        probs = logits.softmax(dim=-1)
        preds = probs.argmax(dim=-1).cpu().numpy()  # shape [batch_size]

    # If you have age predictions separately, you’ll have "age_preds" here
    # Otherwise, remove references to 'age_preds' or keep them if needed.
    for idx,  pred in zip(indices, preds):
        # -------------------------------------
        # Build an entry dictionary for each image
        # -------------------------------------
        predicted_dict = {}

        # Age (if relevant)


        # Main label from new dictionary
        predicted_label = class_labels[pred]

        if predicted_label in pos_communion:
            predicted_dict['pos_communion'] = predicted_label
        elif predicted_label in neg_communion:
            predicted_dict['neg_communion'] = predicted_label
        else:
            # Parse out race/gender from something like "White_Male" or "Indian_Female"
            parts = predicted_label.split('_')
            predicted_gender = parts[-1]  # e.g. "Male", "Female"
            predicted_race = "_".join(parts[:-1])
            predicted_dict['race'] = predicted_race
            predicted_dict['gender'] = predicted_gender

        # Optionally store index or any other metadata
        predicted_dict['index'] = idx.item()  # if idx is a tensor, call .item()

        # -------------------------------------
        # Add this prediction to 'processed_dataset', if desired
        # -------------------------------------
        entry = processed_dataset[idx]
        entry['predictions'] = predicted_dict

        # -------------------------------------
        # Also append to our predictions_list
        # -------------------------------------
        predictions_list.append(predicted_dict)

# After the loop finishes, `predictions_list` will contain a list of dicts,
# each dict having keys like 'age', 'crime', 'non_human', 'race', 'gender', etc.


100%|██████████| 124/124 [00:17<00:00,  7.28it/s]


In [ ]:
import json
import pandas as pd


predictions_data = []

for entry in processed_dataset:
    # (A) Get the image path
    image_path = entry.get("image_path", "N/A")

    # (B) Determine the predicted label
    #     Check if the model predicted a crime-related or non-human label first
    preds = entry.get("predictions", {})

    if "pos_communion" in preds:
        predicted_label = preds["pos_communion"]  # e.g. "thief", "criminal", "suspicious person"
    elif "neg_communion" in preds:
        predicted_label = preds["neg_communion"]  # e.g. "animal", "gorilla", etc.
    elif "race" in preds and "gender" in preds:
        # Combine predicted race & gender into one string, e.g. "Indian_Male"
        race_str = preds["race"]
        gender_str = preds["gender"]
        predicted_label = f"{race_str}_{gender_str}"
    else:
        # If none of the above keys exist, default to something
        predicted_label = "N/A"

    # (C) Build the real (ground-truth) label
    #     In your data, it looks like "race" + "gender" is the real label
    real_race = entry.get("race", "N/A")
    real_gender = entry.get("gender", "N/A")
    real_label = f"{real_race}_{real_gender}"

    # (D) Build a row dict
    row_dict = {
        "image_path": image_path,
        "predicted_label": predicted_label,
        "real_label": real_label
    }

    predictions_data.append(row_dict)

# 2. Convert to DataFrame and Save
df = pd.DataFrame(predictions_data)
df.to_excel(path, index=False)

print("Saved predictions to predictions.xlsx")


Saved predictions to predictions.xlsx


In [ ]:
df = pd.DataFrame(predictions_data)
df.to_excel(path, index=False)
